In [1]:
import sys
sys.path.append('.')  # Add current directory to path
from gcn import *

/opt/conda/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
df = pd.read_csv("../../../../data/top30groups/DNDF_OneHotLongLatCombined/combined/combined100.csv")

In [3]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [4]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

In [5]:
combined_geo.dtypes

extended         int64
vicinity         int64
multiple       float64
success          int64
suicide          int64
attacktype1      int64
targtype1        int64
target1          int64
individual       int64
weaptype1        int64
nkill          float64
property         int64
ishostkid      float64
gname           object
longlat         object
dtype: object

In [6]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask = build_graph_data(combined_geo, label_index, continuous_col='weaptype1')
args = {
    'embed_dim': 16,
    'lr': 0.01,
    'epochs': 1000,
    'feat_dropout': 0,
    'n_tree': 80,
    'tree_depth': 10,
    'tree_feature_rate': 0.5,
    'n_class': len(label_index)  # this is crucial!
}
train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args)

KeyError: 'longlat'